In [1]:
product_name_list = [
 
]

In [2]:
len(product_name_list)

203

In [3]:
def split_list(original_list):
    length = len(original_list)
    size = length // 8
    remainder = length % 8

    sublists = []
    start = 0

    for i in range(8):
        end = start + size + (1 if i < remainder else 0)
        sublists.append(original_list[start:end])
        start = end

    return sublists
sublist = split_list(product_name_list)

In [5]:
len(sublist[0])

26

In [6]:
!pip install -q -U transformers accelerate 

In [7]:
!pip install einops

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 1.7 MB/s eta 0:00:00


In [8]:
import pickle

In [9]:
# # write prompt templates to be passed to GPT-4. more templates allows generating more diverse training data. I give 3 examples here but you should add more.
# # very important to specify the exact number of sentences to generate. 68 is just a number that works for my dataset; you can change it. separate each sentence with a separator so that the sentences can be easily split later.
# # template1 = '''
# # Bạn là khách hàng phàn nàn về việc sản phẩm bạn mua bị thất lạc hoặc hư hỏng. bạn sẽ tạo ra 10 câu. ngăn cách mỗi câu bằng dấu #. chèn # sau dấu câu kết thúc câu. Viết bằng tiếng Việt. 
# # sản phẩm: {product_names}
# # '''

# template2 = '''
# Bạn là một nhà phê bình công nghệ. viết 1 câu về mỗi sản phẩm trong số 10 sản phẩm sau. bạn sẽ tạo ra 10 câu. ngăn cách mỗi câu bằng dấu #. chèn # sau dấu câu kết thúc câu.Viết bằng tiếng Việt.
# sản phẩm: {product_names}
# '''

# # template3 = '''
# # Bạn là khách hàng đang mua sắm sản phẩm. bạn đang nói chuyện với nhân viên hỗ trợ khách hàng trong cuộc trò chuyện trực tuyến. viết một câu hỏi 1 câu về mỗi sản phẩm trong số 10 sản phẩm sau. bạn sẽ tạo ra 10 câu. ngăn cách mỗi câu bằng dấu #. chèn # sau dấu câu kết thúc câu. Viết bằng tiếng Việt.
# # sản phẩm: {product_names}
# # '''
# template4 = '''
# Bạn là một khách hàng hài lòng với sản phẩm bạn đã mua. Bạn sẽ tạo ra 10 câu. Ngăn cách mỗi câu bằng dấu #. Chèn # sau dấu câu kết thúc câu. Viết bằng tiếng Việt.
# sản phẩm: {product_names}
# '''

# template5 = '''
# Bạn là nhân viên bán hàng trực tuyến và bạn đang tư vấn cho khách hàng về các sản phẩm. Bạn sẽ tạo ra 10 câu. Ngăn cách mỗi câu bằng dấu #. Chèn # sau dấu câu kết thúc câu. Viết bằng tiếng Việt.
# sản phẩm: {product_names}
# '''
t1 = '''
Bạn là một nhân viên bán hàng. Khách hàng đang quan tâm đến sản phẩm {product_names}. Bạn hãy thuyết phục khách hàng mua sản phẩm {product_names} qua 10 câu. Ngăn cách mỗi câu bằng dấu #. Chèn # sau dấu câu kết thúc câu. Viết bằng tiếng Việt.
'''
t2 = '''
Bạn là một nhà tiếp thị sản phẩm. Công ty bạn vừa có sản phẩm mới là {product_names}. Bạn hãy viết một bài quảng cáo có 10 câu để giới thiệu về sản phẩm {product_names}. Ngăn cách mỗi câu bằng dấu #. Chèn # sau dấu câu kết thúc câu. Viết bằng tiếng Việt.
'''
t3 = '''
Bạn là một khách hàng đã mua sản phẩm {product_names}. Bạn hãy viết một đoạn văn 10 câu để nêu cảm nghĩ của bản thân sau thời gian sử dụng gửi về cho nhà sản xuất. Ngăn cách mỗi câu bằng dấu #. Chèn # sau dấu câu kết thúc câu. Viết bằng tiếng Việt.
'''

template = [t1,t2,t3]
# template.append(template1)
# template.append(template2)
# template.append(template3)
# template.append(template4)
# template.append(template5)

In [10]:
# Use a pipeline as a high-level helper
from transformers import AutoConfig, AutoModelForCausalLM, AutoTokenizer
import torch
model_path = "vinai/PhoGPT-4B-Chat" 
config = AutoConfig.from_pretrained(model_path, trust_remote_code=True)  
config.init_device = "cuda"
model = AutoModelForCausalLM.from_pretrained(model_path, config=config, torch_dtype=torch.bfloat16, trust_remote_code=True)
model.to('cuda')
model.eval() 
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)

config.json:   0%|          | 0.00/1.19k [00:00<?, ?B/s]

configuration_mpt.py:   0%|          | 0.00/16.4k [00:00<?, ?B/s]

ffn.py:   0%|          | 0.00/5.22k [00:00<?, ?B/s]

fc.py:   0%|          | 0.00/167 [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/vinai/PhoGPT-4B-Chat:
- fc.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/vinai/PhoGPT-4B-Chat:
- ffn.py
- fc.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


...

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


generation_config.json:   0%|          | 0.00/91.0 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/850 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/844k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/96.0 [00:00<?, ?B/s]

In [11]:

def generate_responses(product_names,ins,model,tokenizer): 
    model.to('cuda')
    s = ins.format(product_names=product_names)
    messages = []
    messages.append({"role": "user", "content": s})
    input_prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    input_ids = tokenizer(input_prompt, return_tensors="pt")  
    outputs = model.generate(  
        inputs=input_ids["input_ids"].to("cuda"),  
        attention_mask=input_ids["attention_mask"].to("cuda"),  
        do_sample=True,  
        temperature=1.0,  
        top_k=50,  
        top_p=0.9,  
        max_new_tokens=1024,  
        eos_token_id=tokenizer.eos_token_id,  
        pad_token_id=tokenizer.pad_token_id  
    )  
    response = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]  
    response = response.split("### Trả lời:")[-1]
    return response



In [12]:
from tqdm import tqdm
llm_response_product_name_list = []
total_responses = len(sublist[6]) * len(template) 
print(total_responses)
counter = len(llm_response_product_name_list)

# Tạo tiến trình
progress_bar = tqdm(total=total_responses, initial=counter, desc="Generating responses")

for product_names in sublist[6]:
    for ins in template:
            response = generate_responses(product_names, ins,model,tokenizer)
            if response not in llm_response_product_name_list:
                llm_response_product_name_list.append(response)
                print(response)
                with open('llm_response_product_name_list6.pkl', 'wb') as f:
                    pickle.dump(llm_response_product_name_list, f)
                counter += 1
                progress_bar.update(1)
            # print(f"Generated {counter}/{total_responses} responses")

progress_bar.close()

75


Generating responses:   0%|          | 0/75 [00:00<?, ?it/s]/root/.cache/huggingface/modules/transformers_modules/vinai/PhoGPT-4B-Chat/116013fa63f8c4025739487e1cbff65b7375bbe2/attention.py:87: UserWarning: Propagating key_padding_mask to the attention module and applying it within the attention module can cause unnecessary computation/memory usage. Consider integrating into attn_bias once and passing that to each attention module instead.
  warnings.warn('Propagating key_padding_mask to the attention module ' + 'and applying it within the attention module can cause ' + 'unnecessary computation/memory usage. Consider integrating ' + 'into attn_bias once and passing that to each attention ' + 'module instead.')
Generating responses:   1%|▏         | 1/75 [00:05<06:17,  5.10s/it]

Tôi rất tiếc nhưng tôi không đủ khả năng để cung cấp dịch vụ đó.


Generating responses:   3%|▎         | 2/75 [02:43<1:56:10, 95.49s/it]

Đèn led ốp trần năng lượng mặt trời SUNTEK H05 được thiết kế tinh tế với độ bền cao, khả năng chống thấm nước tốt, tuổi thọ bóng đèn lên đến 50.000 giờ. Đèn có khả năng tự động tắt khi ánh sáng xung quanh tối dần và tự động bật khi có ánh sáng chiếu vào.

- Công suất tối đa của Đèn led ốp trần năng lượng mặt trời SUNTEK H05 là 30W. Vì vậy, khi kết hợp với tấm năng lượng mặt trời, chúng ta sẽ tiết kiệm năng lượng và chi phí cho việc chiếu sáng.

- Đèn led ốp trần năng lượng mặt trời SUNTEK H05 có thiết kế kiểu dáng tròn hiện đại, gọn nhẹ và linh hoạt, phù hợp với nhiều không gian sống như ban công, sân vườn, phòng khách, phòng làm việc, cửa hàng, khách sạn...

- Ngoài ra, Đèn led ốp trần năng lượng mặt trời SUNTEK H05 còn được trang bị pin sạc chất lượng cao, giúp duy trì nguồn điện ổn định và bền bỉ. Thời gian sạc pin chỉ từ 4-6 giờ, và thời gian sử dụng lên đến 8 giờ.

- Với Đèn led ốp trần năng lượng mặt trời SUNTEK H05, bạn sẽ có được không gian sống hiện đại, sang trọng, đồng thời 

...

1. Xin chào, anh/chị đang quan tâm đến sản phẩm Bộ đèn pha năng lượng mặt trời AIO Smart của chúng tôi. Hãy xem mô tả chi tiết và liên hệ với chúng tôi để biết thêm thông tin về sản phẩm nhé.

2. Thưa quý khách hàng, hiện nay trên thị trường có rất nhiều sản phẩm đèn pha năng lượng mặt trời kém chất lượng, gây hại cho người sử dụng. Bộ đèn pha năng lượng mặt trời AIO Smart của chúng tôi cam kết đáp ứng mọi yêu cầu của anh/chị về sản phẩm.

3. Xin chào, sản phẩm Bộ đèn pha năng lượng mặt trời AIO Smart của chúng tôi được thiết kế với công nghệ thông minh, đảm bảo an toàn và tiện ích cho người sử dụng. Anh/chị có thể hoàn toàn yên tâm về sản phẩm.

4. Sản phẩm Bộ đèn pha năng lượng mặt trời AIO Smart của chúng tôi được làm từ những vật liệu cao cấp, không gây độc hại cho người sử dụng và môi trường. Anh/chị có thể yên tâm khi sử dụng sản phẩm.

5. Anh/chị có thể hoàn toàn tin tưởng vào chất lượng sản phẩm Bộ đèn pha năng lượng mặt trời AIO Smart của chúng tôi. Sản phẩm đã được cấp chứng 

Generating responses:  99%|█████████▊| 74/75 [3:07:17<03:35, 215.28s/it]

Tất nhiên rồi! Dưới đây là 10 câu quảng cáo cho Bộ đèn pha năng lượng mặt trời AIO Smart 200W:

Câu 1:
Bộ đèn pha năng lượng mặt trời 200W AIO Smart là lựa chọn chiếu sáng toàn diện cho bất kỳ ngôi nhà nào.

Câu 2:
Bộ đèn pha năng lượng mặt trời AIO Smart 200W có khả năng chống thấm nước và chống bụi tốt.

Câu 3:
Sử dụng đèn pha năng lượng mặt trời 200W AIO Smart của chúng tôi là cách tốt nhất để tiết kiệm năng lượng và tiết kiệm tiền.

Câu 4:
Bộ đèn pha năng lượng mặt trời AIO Smart 200W là sự lựa chọn tiết kiệm chi phí nhất trên thị trường hiện nay.

Câu 5:
Với khả năng tiết kiệm năng lượng lên đến 85% so với bóng đèn truyền thống, bạn có thể tiết kiệm được một khoản tiền đáng kể khi sử dụng Bộ đèn pha năng lượng mặt trời AIO Smart 200W.

Câu 6:
Bộ đèn pha năng lượng mặt trời 200W AIO Smart của chúng tôi là giải pháp hoàn hảo cho những ai đang tìm kiếm giải pháp chiếu sáng tiết kiệm chi phí và đáng tin cậy.

Câu 7:
Đèn pha năng lượng mặt trời AIO Smart 200W là lựa chọn tiết kiệm chi 

Generating responses: 100%|██████████| 75/75 [3:07:59<00:00, 150.39s/it]

Tôi rất hài lòng với sản phẩm Bộ đèn pha năng lượng mặt trời AIO Smart. Công suất 200W. Bảo hành 24 tháng. Tôi đã sử dụng nó hơn một tháng và cảm thấy rất hài lòng. Đèn có độ sáng cao, chiếu sáng tốt và bền bỉ. Đèn được thiết kế nhỏ gọn và dễ lắp đặt. Tôi đã rất bất ngờ với độ sáng của đèn và tôi đã có thể dễ dàng nhìn rõ mọi thứ trong phạm vi chiếu sáng của đèn. Tôi rất ấn tượng với chính sách bảo hành của đèn, nó rất tốt và đáng tin cậy. Tôi rất muốn giới thiệu sản phẩm này cho những người khác và hy vọng rằng họ cũng sẽ có trải nghiệm tích cực như tôi.
